In [1]:
import os
import pandas as pd
import numpy as np
import re
import dask.dataframe as dd

path = 'C:/Users/user/Desktop/AIML/datasets/Data_Preprocessing/spam_set'
files = os.listdir(path)
filenames = []

for root, dirs, files in os.walk(path):
  for file in files:
    if file.endswith('.csv'):
      filenames.append(os.path.join(root, file))

# Create a list to store DataFrames
all_dfs = []

for file in filenames:
  try:
    # Use 'on_bad_lines' instead of 'error_bad_lines'
    # df = pd.read_csv(file, on_bad_lines='skip', delimiter=",")  # Skip bad lines
    df = pd.read_csv(file)

    # 정상화
    df.columns=['index', 'data']
    df = df.set_index('index')
    df['spam'] = 1
    df['data'] = df['data'].str.replace('ifg@','')

    # Dask Dataframe 생성
    # Move ddf creation after modifying the Pandas DataFrame
    ddf = dd.from_pandas(df, npartitions=4) # Create ddf after modifications


    # Append the current DataFrame to the list
    result = ddf.compute()
    all_dfs.append(result)

  except pd.errors.ParserError as e:
    print(f"Error reading file {file}: {e}")

# Concatenate all DataFrames in the list into a single DataFrame
raw = pd.concat(all_dfs, ignore_index=True)

In [2]:
# 텍스트 처리 함수 설정
def preprocess_text(text):
    # 여러 개의 공백을 하나의 공백으로 축소
    text = re.sub(r'\s+', ' ', text)
    # 앞뒤 공백 제거
    text = text.strip()
    # 맨 앞의 숫자와 공백 또는 콜론 제거
    text = re.sub(r'^\d+\s*', '', text)
    # 필요에 따라 특수 문자 제거 (예시로 일부 문자만 제거)
    text = re.sub(r'[^가-힣a-zA-Z0-9\s]', '', text)
    return text

In [3]:
# 결측치 처리
print(len(raw))
drop_index = raw[raw['data'].isnull()].index
raw = raw.drop(drop_index)
print(len(raw))

50639
50639


In [4]:
# 중복 내용 제거
print(len(raw))
raw = raw.drop_duplicates('data')
print(len(raw))

50639
49786


In [5]:
raw['data']=raw['data'].apply(preprocess_text)

In [6]:
raw['data'] = raw['data'].str.replace('Web발신', '')
raw['data'] = raw['data'].str.replace('광고', '')

In [7]:
raw.head()

,data,spam
0,님 작전주에 당첨되셨습니다이 문자는 분뒤 삭제됩니다방입장시 종목부터 즉시공개 하겠...,1
1,국제발신처음 치킨cupon새벽 방문만humluck 쩜컴CDhbq,1
2,국제발신이오스 엔지엔 에볼루정답쥐조작쥐후불제결과로보답합니다백프로입니다엔트리안내comFW,1
3,국제발신회 오 리 커피값 시시회 오 리가 아니면 사 칭사칭 먹튀에 주의 바랍니다툭 HOL,1
4,국외발신새고객 원출금해 가세요 십만원샌즈 WRi계열 새해 GIFTasddcom,1


In [8]:
raw.to_csv('C:/Users/user/Desktop/AIML/datasets/Data_Preprocessing/spam_data.csv', index=False)